In [9]:
import os, sys
sys.path.insert(0, '..')

import glob
import joblib
from collections import defaultdict
from itertools import product

import pandas as pd
from tqdm import tqdm

from src.data.io import read_df, write_df
from src.data.utils import round_to_significant
from src.ml.evaluate import inner_score
from src.analysis.aggregate import scores_to_df, study_to_df
from src.analysis.stats import *

/Users/miwan/miniforge3/envs/pmd/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Aggregate and process results from model training

In [ ]:
"""
Aggregated results (aggregated.tar.gz) are provided in the same directory as raw dataset
and can be unpacked using:
tar -xvf aggregated.tar.gz
"""

In [9]:
global_dir = '../results/training'

In [3]:
# Combine files made for each fold

ddc = defaultdict(int)

for file in tqdm(glob.glob(f'{global_dir}/*/*/*/*.joblib'), total=1620*4):
    _, _, _, dataset_pt_dpa, model, descriptors, file_name = file.split('/')

    if 'ensemble' in file_name:
        continue

    dataset, pt, dpa = dataset_pt_dpa.split('_')
    _, _, test_fold = file_name.rstrip('.joblib').split('_')
    df = read_df(file)

    if 'preds' in file_name:
        df['Signature'] = df['Signature'].apply(lambda sign: ' | '.join(sign))
        df['Test_Fold'] = test_fold
        out_path = file.replace('.joblib', '.tsv')
        write_df(df, out_path)

    elif 'scores' in file_name:
        df = scores_to_df(df)
        df['Test_Fold'] = test_fold
        out_path = file.replace('.joblib', '.tsv')
        write_df(df, out_path)

    elif 'study' in file_name:
        df = study_to_df(df)
        df['Test_Fold'] = test_fold
        df['Model'] = model
        df['Descriptors'] = descriptors
        out_path = file.replace('.joblib', '.tsv')
        write_df(df, out_path)

    else:
        raise ValueError('File not found, please check your code :c')

  5%|▌         | 243/4860 [00:04<01:22, 55.74it/s]/wuppertal/gptil/Repos/PMD/src/analysis/aggregate.py:68: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  study_df = pd.concat(trial_dfs).reset_index(drop=True)
  5%|▌         | 267/4860 [00:04<01:25, 53.61it/s]/wuppertal/gptil/Repos/PMD/src/analysis/aggregate.py:68: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  study_df = pd.concat(trial_dfs).reset_index(drop=True)
 13%|█▎        | 609/4860 [00:11<01:17, 54.72it/s]/wuppertal/gptil/Repos/PMD/src/ana

In [4]:
# Combine scores into single file on a per dataset_variant-model-descriptor basis

ddc_scores = defaultdict(list)

for file in tqdm(glob.glob('../results/training/*/*/*/scores*.tsv'), total=1620):
    _, _, _, dataset_pt_dpa, model, descriptors, file_name = file.split('/')
    dataset, pt, dpa = dataset_pt_dpa.split('_')
    _, _, test_fold = file_name.rstrip('.tsv').split('_')
    df = read_df(file)
    key = (dataset, pt, dpa, model, descriptors)
    ddc_scores[key].append(df)

for (dataset, pt, dpa, model, descriptors), values in ddc_scores.items():
    os.makedirs(f'../results/aggregated/{dataset}_{pt}_{dpa}/', exist_ok=True)
    out_path = f'../results/aggregated/{dataset}_{pt}_{dpa}/{model}_{descriptors}_scores.tsv'
    df = pd.concat(values).reset_index(drop=True)
    df['Model'] = model
    df['Descriptors'] = descriptors
    write_df(df, out_path)

100%|██████████| 1620/1620 [00:01<00:00, 817.49it/s]


In [5]:
# Combine studies into single file on a per dataset_variant-model-descriptor basis

ddc_scores = defaultdict(list)

for file in tqdm(glob.glob('../results/training/*/*/*/study*.tsv'), total=1620):
    _, _, _, dataset_pt_dpa, model, descriptors, file_name = file.split('/')
    dataset, pt, dpa = dataset_pt_dpa.split('_')
    _, _, test_fold = file_name.rstrip('.tsv').split('_')
    df = read_df(file)
    key = (dataset, pt, dpa, model, descriptors)
    ddc_scores[key].append(df)

for (dataset, pt, dpa, model, descriptors), values in ddc_scores.items():
    os.makedirs(f'../results/aggregated/{dataset}_{pt}_{dpa}/', exist_ok=True)
    out_path = f'../results/aggregated/{dataset}_{pt}_{dpa}/{model}_{descriptors}_studies.tsv'
    df = pd.concat(values).reset_index(drop=True)
    write_df(df, out_path)

100%|██████████| 1620/1620 [00:02<00:00, 808.08it/s]


In [6]:
# Combine predictions into single file on a per dataset_variant-model-descriptor basis

ddc_scores = defaultdict(list)

for file in tqdm(glob.glob('../results/training/*/*/*/preds*.tsv'), total=1620):
    _, _, _, dataset_pt_dpa, model, descriptors, file_name = file.split('/')
    dataset, pt, dpa = dataset_pt_dpa.split('_')
    _, _, test_fold = file_name.rstrip('.tsv').split('_')
    df = read_df(file)
    key = (dataset, pt, dpa, model, descriptors)
    ddc_scores[key].append(df)

for (dataset, pt, dpa, model, descriptors), values in tqdm(ddc_scores.items(), total=324):
    os.makedirs(f'../results/aggregated/{dataset}_{pt}_{dpa}/', exist_ok=True)
    out_path = f'../results/aggregated/{dataset}_{pt}_{dpa}/{model}_{descriptors}_preds.tsv'
    df = pd.concat(values).reset_index(drop=True)
    df['Model'] = model
    df['Descriptors'] = descriptors
    write_df(df, out_path)

100%|██████████| 324/324 [00:56<00:00,  5.70it/s]


### Evaluate Inclusion Criteria

In [30]:
# Aggregate results by inclusion criteria and score on overlapping parts of corresponding datasets

pts = ["Cred", "Card", "Cvas"]
dpas = ["prr", "ror", "ic"]
models = ["LogisticRegression", "RandomForestClassifier", "XGBClassifier"]
descriptors = ["CDDD", "MACCS", "Klek", "RDKit", "ChemBERTa", "Morgan"]

for pt, dpa, model, desc in tqdm(product(pts, dpas, models, descriptors), total=3*3*3*6):

    primary_path = f'../results/aggregated/primary_{pt}_{dpa}/{model}_{desc}_preds.tsv'
    secondary_path = f'../results/aggregated/secondary_{pt}_{dpa}/{model}_{desc}_preds.tsv'

    primary_df = read_df(primary_path)
    secondary_df = read_df(secondary_path)

    primary_df = primary_df.rename(columns={
        'y_pred': 'y_pred_primary',
        'y_score': 'y_score_primary',
    })

    secondary_df = secondary_df.rename(columns={
        'y_pred': 'y_pred_secondary',
        'y_score': 'y_score_secondary',
    })

    overlap_df = pd.merge(primary_df, secondary_df, on=['SMILES', 'Signature', 'Test_Fold', 'Model', 'Descriptors', 'y_true'])

    primary_ddc = defaultdict(list)
    secondary_ddc = defaultdict(list)

    for fold in overlap_df['Test_Fold'].unique():

        sub_overlap_df = overlap_df[overlap_df['Test_Fold'] == fold].reset_index(drop=True)

        primary_score = inner_score(
            y_true=sub_overlap_df['y_true'],
            y_pred=sub_overlap_df['y_pred_primary'],
            y_score=sub_overlap_df['y_score_primary']
        )

        secondary_score = inner_score(
            y_true=sub_overlap_df['y_true'],
            y_pred=sub_overlap_df['y_pred_secondary'],
            y_score=sub_overlap_df['y_score_secondary']
        )

        for key, value in primary_score.items():
            primary_ddc[key].append(value)

        for key, value in secondary_score.items():
            secondary_ddc[key].append(value)

        primary_ddc['Test_Fold'].append(fold)
        secondary_ddc['Test_Fold'].append(fold)

    primary_scores_df = pd.DataFrame(primary_ddc)
    secondary_scores_df = pd.DataFrame(secondary_ddc)

    primary_scores_df['Dataset'] = 'Primary'
    secondary_scores_df['Dataset'] = 'Secondary'

    scores_df = pd.concat([primary_scores_df, secondary_scores_df])
    scores_df['PT'] = pt.capitalize()
    scores_df['DPA'] = dpa.upper()
    scores_df['Model'] = model
    scores_df['Descriptors'] = desc

    melt_df = pd.melt(
        frame=scores_df, id_vars=['Dataset', 'DPA', 'PT', 'Test_Fold', 'Model', 'Descriptors'],
        value_vars=['Balanced Accuracy', 'GeomRS', 'HarmRS', 'F1 Score', 'ROC AUC', 'MCC', 'Accuracy', 'Recall', 'Specificity', 'Precision'],
        var_name='Metric', value_name='Value')

    out_dir = f'../results/comparison/primary_secondary'
    os.makedirs(out_dir, exist_ok=True)

    out_path = os.path.join(out_dir, f'{pt}_{dpa}_{model}_{desc}_scores.tsv')
    write_df(melt_df, out_path)

100%|██████████| 162/162 [00:10<00:00, 15.26it/s]


In [31]:
# Format to a table

dfs = []
for file in glob.glob('../results/comparison/primary_secondary/*.tsv'):
    dfs.append(read_df(file))
df = pd.concat(dfs).reset_index(drop=True)

mean_df = df.groupby(by=['Dataset', 'Descriptors', 'Metric', 'Model'])['Value'].mean().reset_index(name='Mean')
std_df = df.groupby(by=['Dataset', 'Descriptors', 'Metric', 'Model'])['Value'].std().reset_index(name='STD')

comb_df = mean_df.merge(std_df, on=['Dataset', 'Descriptors', 'Metric', 'Model'])
comb_df['Mean'] = comb_df['Mean'].apply(lambda value: np.round(value, 5))
comb_df['STD'] = comb_df['STD'].apply(lambda value: np.round(value, 5))

write_df(comb_df, '../results/comparison/dataset_type_detailed.tsv')

In [32]:
dfs = []
for file in glob.glob('../results/comparison/primary_secondary/*.tsv'):
    dfs.append(read_df(file))
df = pd.concat(dfs).reset_index(drop=True)

mean_df = df.groupby(by=['Dataset', 'Metric'])['Value'].mean().reset_index(name='Mean')
std_df = df.groupby(by=['Dataset', 'Metric'])['Value'].std().reset_index(name='STD')

comb_df = mean_df.merge(std_df, on=['Dataset', 'Metric'])
comb_df['Mean'] = comb_df['Mean'].apply(lambda value: np.round(value, 5))
comb_df['STD'] = comb_df['STD'].apply(lambda value: np.round(value, 5))

write_df(comb_df, '../results/comparison/dataset_type.tsv')

In [33]:
# Print as LaTeX rows
df = read_df('../results/comparison/dataset_type_detailed.tsv')

for dataset in ['Primary', 'Secondary']:

    ds_line = "\multirow[t]{18}{*}{" + dataset + "}"
    print(ds_line)

    for model in ['LogisticRegression', 'RandomForestClassifier', 'XGBClassifier']:

        ml_line = "& \multirow[t]{6}{*}{" + model + "}"
        print(ml_line)

        sub_df = pl.from_pandas(df).filter((pl.col('Dataset') == dataset) & (pl.col('Model') == model))
        sub_df = sub_df.with_columns([
            (pl.col('Mean') * 100).round(0).cast(pl.Int16).alias('Mean'),
            (pl.col('STD') * 100).round(0).cast(pl.Int16).alias('STD'),
        ])

        for idx, desc in enumerate(['CDDD', 'ChemBERTa', 'Klek', 'MACCS', 'Morgan', 'RDKit']):
            metrics = sub_df.filter(pl.col('Descriptors') == desc)
            rc_mean = metrics.filter(pl.col('Metric') == 'Recall')['Mean'].item()
            rc_std = metrics.filter(pl.col('Metric') == 'Recall')['STD'].item()
            sp_mean = metrics.filter(pl.col('Metric') == 'Specificity')['Mean'].item()
            sp_std = metrics.filter(pl.col('Metric') == 'Specificity')['STD'].item()
            roc_mean = metrics.filter(pl.col('Metric') == 'ROC AUC')['Mean'].item()
            roc_std = metrics.filter(pl.col('Metric') == 'ROC AUC')['STD'].item()
            harm_mean = metrics.filter(pl.col('Metric') == 'HarmRS')['Mean'].item()
            harm_std = metrics.filter(pl.col('Metric') == 'HarmRS')['STD'].item()

            if idx == 0:
                pre = "  &"
            else:
                pre = "& &"

            empty_lines = " " * (9 - len(desc))

            line = fr"{pre} {desc} {empty_lines}& {rc_mean} ± {rc_std} & {sp_mean} ± {sp_std} & {roc_mean} ± {roc_std} & {harm_mean} ± {harm_std} \\"
            print(line)

\multirow[t]{18}{*}{Primary}
& \multirow[t]{6}{*}{LogisticRegression}
  & CDDD      & 57 ± 7 & 52 ± 8 & 56 ± 3 & 53 ± 3 \\
& & ChemBERTa & 53 ± 8 & 50 ± 7 & 52 ± 3 & 51 ± 2 \\
& & Klek      & 54 ± 9 & 57 ± 8 & 57 ± 3 & 54 ± 3 \\
& & MACCS     & 53 ± 14 & 56 ± 15 & 56 ± 2 & 51 ± 4 \\
& & Morgan    & 58 ± 9 & 56 ± 10 & 59 ± 4 & 56 ± 4 \\
& & RDKit     & 54 ± 14 & 58 ± 13 & 58 ± 2 & 53 ± 4 \\
& \multirow[t]{6}{*}{RandomForestClassifier}
  & CDDD      & 65 ± 12 & 49 ± 13 & 62 ± 3 & 53 ± 6 \\
& & ChemBERTa & 63 ± 13 & 46 ± 12 & 58 ± 4 & 51 ± 5 \\
& & Klek      & 55 ± 18 & 57 ± 18 & 60 ± 3 & 50 ± 7 \\
& & MACCS     & 64 ± 11 & 51 ± 10 & 61 ± 3 & 55 ± 3 \\
& & Morgan    & 54 ± 18 & 61 ± 18 & 62 ± 3 & 51 ± 7 \\
& & RDKit     & 65 ± 11 & 50 ± 9 & 62 ± 4 & 55 ± 3 \\
& \multirow[t]{6}{*}{XGBClassifier}
  & CDDD      & 62 ± 11 & 54 ± 12 & 62 ± 2 & 56 ± 4 \\
& & ChemBERTa & 62 ± 12 & 49 ± 12 & 59 ± 3 & 52 ± 5 \\
& & Klek      & 58 ± 13 & 56 ± 13 & 61 ± 4 & 54 ± 5 \\
& & MACCS     & 60 ± 13 & 57 ± 1

### Evaluate DPA metric

In [11]:
# Aggregate results by DPA metric and score on overlapping parts of corresponding datasets

datasets = ['primary', 'secondary']
dpas = ["prr", "ror", "ic"]
pts = ["Cred", "Card", "Cvas"]
models = ["LogisticRegression", "RandomForestClassifier", "XGBClassifier"]
descriptors = ["CDDD", "MACCS", "Klek", "RDKit", "ChemBERTa", "Morgan"]

for dataset, pt, model, desc in tqdm(product(datasets, pts, models, descriptors), total=2*3*3*6):

    cred_path = f'../results/aggregated/{dataset}_{pt}_prr/{model}_{desc}_preds.tsv'
    card_path = f'../results/aggregated/{dataset}_{pt}_ror/{model}_{desc}_preds.tsv'
    cvas_path = f'../results/aggregated/{dataset}_{pt}_ic/{model}_{desc}_preds.tsv'

    cred_df = read_df(cred_path)
    card_df = read_df(card_path)
    cvas_df = read_df(cvas_path)

    cred_df = cred_df.rename(columns={
        'y_pred': 'y_pred_prr',
        'y_score': 'y_score_prr',
    })

    card_df = card_df.rename(columns={
        'y_pred': 'y_pred_ror',
        'y_score': 'y_score_ror',
    })

    cvas_df = cvas_df.rename(columns={
        'y_pred': 'y_pred_ic',
        'y_score': 'y_score_ic',
    })

    merge_on = ['SMILES', 'Signature', 'Test_Fold', 'Model', 'Descriptors', 'y_true']

    overlap_df = pd.merge(cred_df, card_df, on=merge_on, how='inner').merge(cvas_df, on=merge_on, how='inner')

    prr_ddc = defaultdict(list)
    ror_ddc = defaultdict(list)
    ic_ddc = defaultdict(list)

    for fold in overlap_df['Test_Fold'].unique():

        sub_overlap_df = overlap_df[overlap_df['Test_Fold'] == fold].reset_index(drop=True)

        prr_score = inner_score(
            y_true=sub_overlap_df['y_true'],
            y_pred=sub_overlap_df['y_pred_prr'],
            y_score=sub_overlap_df['y_score_prr']
        )

        ror_score = inner_score(
            y_true=sub_overlap_df['y_true'],
            y_pred=sub_overlap_df['y_pred_ror'],
            y_score=sub_overlap_df['y_score_ror']
        )

        ic_score = inner_score(
            y_true=sub_overlap_df['y_true'],
            y_pred=sub_overlap_df['y_pred_ic'],
            y_score=sub_overlap_df['y_score_ic']
        )

        for key, value in prr_score.items():
            prr_ddc[key].append(value)

        for key, value in ror_score.items():
            ror_ddc[key].append(value)

        for key, value in ic_score.items():
            ic_ddc[key].append(value)

        prr_ddc['Test_Fold'].append(fold)
        ror_ddc['Test_Fold'].append(fold)
        ic_ddc['Test_Fold'].append(fold)

    prr_scores_df = pd.DataFrame(prr_ddc)
    ror_scores_df = pd.DataFrame(ror_ddc)
    ic_scores_df = pd.DataFrame(ic_ddc)

    prr_scores_df['DPA'] = 'PRR'
    ror_scores_df['DPA'] = 'ROR'
    ic_scores_df['DPA'] = 'IC'

    scores_df = pd.concat([prr_scores_df, ror_scores_df, ic_scores_df])
    scores_df['Dataset'] = dataset.capitalize()
    scores_df['PT'] = pt.capitalize()
    scores_df['Model'] = model
    scores_df['Descriptors'] = desc

    melt_df = pd.melt(
        frame=scores_df, id_vars=['Dataset', 'DPA', 'PT', 'Test_Fold', 'Model', 'Descriptors'],
        value_vars=['Balanced Accuracy', 'GeomRS', 'HarmRS', 'F1 Score', 'ROC AUC', 'MCC', 'Accuracy', 'Recall', 'Specificity', 'Precision'],
        var_name='Metric', value_name='Value')

    out_dir = f'../results/comparison/prr_ror_ic'
    os.makedirs(out_dir, exist_ok=True)

    out_path = os.path.join(out_dir, f'{dataset}_{pt}_{model}_{desc}_scores.tsv')
    write_df(melt_df, out_path)

100%|██████████| 108/108 [00:11<00:00,  9.49it/s]


In [35]:
dfs = []
for file in glob.glob('../results/comparison/prr_ror_ic/*.tsv'):
    dfs.append(read_df(file))
df = pd.concat(dfs).reset_index(drop=True)

mean_df = df.groupby(by=['DPA', 'Descriptors', 'Metric', 'Model'])['Value'].mean().reset_index(name='Mean')
std_df = df.groupby(by=['DPA', 'Descriptors', 'Metric', 'Model'])['Value'].std().reset_index(name='STD')

comb_df = mean_df.merge(std_df, on=['DPA', 'Descriptors', 'Metric', 'Model'])
comb_df['Mean'] = comb_df['Mean'].apply(lambda value: np.round(value, 5))
comb_df['STD'] = comb_df['STD'].apply(lambda value: np.round(value, 5))

write_df(comb_df, '../results/comparison/dpa_metric_detailed.tsv')

In [36]:
dfs = []
for file in glob.glob('../results/comparison/prr_ror_ic/*.tsv'):
    dfs.append(read_df(file))
df = pd.concat(dfs).reset_index(drop=True)

mean_df = df.groupby(by=['DPA', 'Metric'])['Value'].mean().reset_index(name='Mean')
std_df = df.groupby(by=['DPA', 'Metric'])['Value'].std().reset_index(name='STD')

comb_df = mean_df.merge(std_df, on=['DPA', 'Metric'])
comb_df['Mean'] = comb_df['Mean'].apply(lambda value: np.round(value, 5))
comb_df['STD'] = comb_df['STD'].apply(lambda value: np.round(value, 5))

write_df(comb_df, '../results/comparison/dpa_metric.tsv')

In [37]:
# Print as LaTeX rows
df = read_df('../results/comparison/dpa_metric_detailed.tsv')

for dpa_metric in ['PRR', 'ROR', 'IC']:

    ds_line = "\multirow[t]{18}{*}{" + dpa_metric + "}"
    print(ds_line)

    for model in ['LogisticRegression', 'RandomForestClassifier', 'XGBClassifier']:

        ml_line = "& \multirow[t]{6}{*}{" + model + "}"
        print(ml_line)

        sub_df = pl.from_pandas(df).filter((pl.col('DPA') == dpa_metric) & (pl.col('Model') == model))
        sub_df = sub_df.with_columns([
            (pl.col('Mean') * 100).round(0).cast(pl.Int16).alias('Mean'),
            (pl.col('STD') * 100).round(0).cast(pl.Int16).alias('STD'),
        ])

        for idx, desc in enumerate(['CDDD', 'ChemBERTa', 'Klek', 'MACCS', 'Morgan', 'RDKit']):
            metrics = sub_df.filter(pl.col('Descriptors') == desc)
            rc_mean = metrics.filter(pl.col('Metric') == 'Recall')['Mean'].item()
            rc_std = metrics.filter(pl.col('Metric') == 'Recall')['STD'].item()
            sp_mean = metrics.filter(pl.col('Metric') == 'Specificity')['Mean'].item()
            sp_std = metrics.filter(pl.col('Metric') == 'Specificity')['STD'].item()
            roc_mean = metrics.filter(pl.col('Metric') == 'ROC AUC')['Mean'].item()
            roc_std = metrics.filter(pl.col('Metric') == 'ROC AUC')['STD'].item()
            harm_mean = metrics.filter(pl.col('Metric') == 'HarmRS')['Mean'].item()
            harm_std = metrics.filter(pl.col('Metric') == 'HarmRS')['STD'].item()

            if idx == 0:
                pre = "  &"
            else:
                pre = "& &"

            empty_lines = " " * (9 - len(desc))

            line = fr"{pre} {desc} {empty_lines}& {rc_mean} ± {rc_std} & {sp_mean} ± {sp_std} & {roc_mean} ± {roc_std} & {harm_mean} ± {harm_std} \\"
            print(line)

\multirow[t]{18}{*}{PRR}
& \multirow[t]{6}{*}{LogisticRegression}
  & CDDD      & 54 ± 4 & 52 ± 7 & 55 ± 3 & 53 ± 2 \\
& & ChemBERTa & 51 ± 7 & 53 ± 6 & 52 ± 3 & 51 ± 2 \\
& & Klek      & 52 ± 7 & 58 ± 4 & 57 ± 3 & 54 ± 3 \\
& & MACCS     & 50 ± 6 & 59 ± 5 & 56 ± 2 & 54 ± 2 \\
& & Morgan    & 53 ± 8 & 57 ± 8 & 57 ± 3 & 54 ± 2 \\
& & RDKit     & 52 ± 6 & 59 ± 4 & 57 ± 2 & 55 ± 2 \\
& \multirow[t]{6}{*}{RandomForestClassifier}
  & CDDD      & 62 ± 8 & 48 ± 11 & 58 ± 2 & 53 ± 5 \\
& & ChemBERTa & 61 ± 11 & 45 ± 11 & 54 ± 3 & 50 ± 4 \\
& & Klek      & 53 ± 11 & 57 ± 14 & 58 ± 2 & 52 ± 10 \\
& & MACCS     & 57 ± 9 & 54 ± 8 & 57 ± 3 & 54 ± 3 \\
& & Morgan    & 53 ± 9 & 61 ± 10 & 59 ± 3 & 55 ± 3 \\
& & RDKit     & 63 ± 9 & 47 ± 8 & 58 ± 2 & 53 ± 3 \\
& \multirow[t]{6}{*}{XGBClassifier}
  & CDDD      & 58 ± 7 & 53 ± 7 & 58 ± 1 & 55 ± 2 \\
& & ChemBERTa & 58 ± 8 & 50 ± 8 & 55 ± 2 & 52 ± 2 \\
& & Klek      & 53 ± 6 & 59 ± 4 & 58 ± 3 & 55 ± 3 \\
& & MACCS     & 56 ± 6 & 57 ± 4 & 59 ± 2 & 56 ± 2 \

### Evaluate Cardiotoxicity Definition

In [38]:
# Aggregate results by cardiotoxicity definition and score on overlapping parts of corresponding datasets

datasets = ['primary', 'secondary']
dpas = ["prr", "ror", "ic"]
models = ["LogisticRegression", "RandomForestClassifier", "XGBClassifier"]
descriptors = ["CDDD", "MACCS", "Klek", "RDKit", "ChemBERTa", "Morgan"]

for dataset, dpa, model, desc in tqdm(product(datasets, dpas, models, descriptors), total=2*3*3*6):

    cred_path = f'../results/aggregated/{dataset}_Cred_{dpa}/{model}_{desc}_preds.tsv'
    card_path = f'../results/aggregated/{dataset}_Card_{dpa}/{model}_{desc}_preds.tsv'
    cvas_path = f'../results/aggregated/{dataset}_Cvas_{dpa}/{model}_{desc}_preds.tsv'

    cred_df = read_df(cred_path)
    card_df = read_df(card_path)
    cvas_df = read_df(cvas_path)

    cred_df = cred_df.rename(columns={
        'y_pred': 'y_pred_cred',
        'y_score': 'y_score_cred',
    })

    card_df = card_df.rename(columns={
        'y_pred': 'y_pred_card',
        'y_score': 'y_score_card',
    })

    cvas_df = cvas_df.rename(columns={
        'y_pred': 'y_pred_cvas',
        'y_score': 'y_score_cvas',
    })

    merge_on = ['SMILES', 'Signature', 'Test_Fold', 'Model', 'Descriptors', 'y_true']

    overlap_df = pd.merge(cred_df, card_df, on=merge_on, how='inner').merge(cvas_df, on=merge_on, how='inner')

    cred_ddc = defaultdict(list)
    card_ddc = defaultdict(list)
    cvas_ddc = defaultdict(list)

    for fold in overlap_df['Test_Fold'].unique():

        sub_overlap_df = overlap_df[overlap_df['Test_Fold'] == fold].reset_index(drop=True)

        cred_score = inner_score(
            y_true=sub_overlap_df['y_true'],
            y_pred=sub_overlap_df['y_pred_cred'],
            y_score=sub_overlap_df['y_score_cred']
        )

        card_score = inner_score(
            y_true=sub_overlap_df['y_true'],
            y_pred=sub_overlap_df['y_pred_card'],
            y_score=sub_overlap_df['y_score_card']
        )

        cvas_score = inner_score(
            y_true=sub_overlap_df['y_true'],
            y_pred=sub_overlap_df['y_pred_cvas'],
            y_score=sub_overlap_df['y_score_cvas']
        )

        for key, value in cred_score.items():
            cred_ddc[key].append(value)

        for key, value in card_score.items():
            card_ddc[key].append(value)

        for key, value in cvas_score.items():
            cvas_ddc[key].append(value)

        cred_ddc['Test_Fold'].append(fold)
        card_ddc['Test_Fold'].append(fold)
        cvas_ddc['Test_Fold'].append(fold)

    cred_scores_df = pd.DataFrame(cred_ddc)
    card_scores_df = pd.DataFrame(card_ddc)
    cvas_scores_df = pd.DataFrame(cvas_ddc)

    cred_scores_df['PT'] = 'Cred'
    card_scores_df['PT'] = 'Card'
    cvas_scores_df['PT'] = 'Cvas'

    scores_df = pd.concat([cred_scores_df, card_scores_df, cvas_scores_df])
    scores_df['Dataset'] = dataset.capitalize()
    scores_df['DPA'] = dpa.upper()
    scores_df['Model'] = model
    scores_df['Descriptors'] = desc

    melt_df = pd.melt(
        frame=scores_df, id_vars=['Dataset', 'DPA', 'PT', 'Test_Fold', 'Model', 'Descriptors'],
        value_vars=['Balanced Accuracy', 'GeomRS', 'HarmRS', 'F1 Score', 'ROC AUC', 'MCC', 'Accuracy', 'Recall', 'Specificity', 'Precision'],
        var_name='Metric', value_name='Value')

    out_dir = f'../results/comparison/cred_card_cvas'
    os.makedirs(out_dir, exist_ok=True)

    out_path = os.path.join(out_dir, f'{dataset}_{dpa}_{model}_{desc}_scores.tsv')
    write_df(melt_df, out_path)

100%|██████████| 108/108 [00:11<00:00,  9.78it/s]


Values for tables

In [39]:
dfs = []
for file in glob.glob('../results/comparison/cred_card_cvas/*.tsv'):
    dfs.append(read_df(file))
df = pd.concat(dfs).reset_index(drop=True)

mean_df = df.groupby(by=['PT', 'Descriptors', 'Metric', 'Model'])['Value'].mean().reset_index(name='Mean')
std_df = df.groupby(by=['PT', 'Descriptors', 'Metric', 'Model'])['Value'].std().reset_index(name='STD')

comb_df = mean_df.merge(std_df, on=['PT', 'Descriptors', 'Metric', 'Model'])
comb_df['Mean'] = comb_df['Mean'].apply(lambda value: np.round(value, 5))
comb_df['STD'] = comb_df['STD'].apply(lambda value: np.round(value, 5))

write_df(comb_df, '../results/comparison/pt_set_detailed.tsv')

In [41]:
dfs = []
for file in glob.glob('../results/comparison/cred_card_cvas/*.tsv'):
    dfs.append(read_df(file))
df = pd.concat(dfs).reset_index(drop=True)

mean_df = df.groupby(by=['PT', 'Metric'])['Value'].mean().reset_index(name='Mean')
std_df = df.groupby(by=['PT', 'Metric'])['Value'].std().reset_index(name='STD')

comb_df = mean_df.merge(std_df, on=['PT', 'Metric'])
comb_df['Mean'] = comb_df['Mean'].apply(lambda value: np.round(value, 5))
comb_df['STD'] = comb_df['STD'].apply(lambda value: np.round(value, 5))

write_df(comb_df, '../results/comparison/pt_set.tsv')

In [42]:
# Print as LaTeX rows
df = read_df('../results/comparison/pt_set_detailed.tsv')

for pt_set in ['Cred', 'Card', 'Cvas']:

    ds_line = "\t\multirow[t]{18}{*}{" + pt_set + "}"
    print(ds_line)

    for model in ['LogisticRegression', 'RandomForestClassifier', 'XGBClassifier']:

        ml_line = "\t\t& \multirow[t]{6}{*}{" + model + "}"
        print(ml_line)

        sub_df = pl.from_pandas(df).filter((pl.col('PT') == pt_set) & (pl.col('Model') == model))
        sub_df = sub_df.with_columns([
            (pl.col('Mean') * 100).round(0).cast(pl.Int16).alias('Mean'),
            (pl.col('STD') * 100).round(0).cast(pl.Int16).alias('STD'),
        ])

        for idx, desc in enumerate(['CDDD', 'ChemBERTa', 'Klek', 'MACCS', 'Morgan', 'RDKit']):
            metrics = sub_df.filter(pl.col('Descriptors') == desc)
            rc_mean = metrics.filter(pl.col('Metric') == 'Recall')['Mean'].item()
            rc_std = metrics.filter(pl.col('Metric') == 'Recall')['STD'].item()
            sp_mean = metrics.filter(pl.col('Metric') == 'Specificity')['Mean'].item()
            sp_std = metrics.filter(pl.col('Metric') == 'Specificity')['STD'].item()
            roc_mean = metrics.filter(pl.col('Metric') == 'ROC AUC')['Mean'].item()
            roc_std = metrics.filter(pl.col('Metric') == 'ROC AUC')['STD'].item()
            harm_mean = metrics.filter(pl.col('Metric') == 'HarmRS')['Mean'].item()
            harm_std = metrics.filter(pl.col('Metric') == 'HarmRS')['STD'].item()

            if idx == 0:
                pre = "  &"
            else:
                pre = "& &"

            empty_lines = " " * (9 - len(desc))

            line = f"\t\t\t{pre} {desc} {empty_lines}& {rc_mean} ± {rc_std} & {sp_mean} ± {sp_std} & {roc_mean} ± {roc_std} & {harm_mean} ± {harm_std} \\\\"
            print(line)

	\multirow[t]{18}{*}{Cred}
		& \multirow[t]{6}{*}{LogisticRegression}
			  & CDDD      & 57 ± 8 & 51 ± 10 & 56 ± 3 & 52 ± 4 \\
			& & ChemBERTa & 53 ± 8 & 50 ± 8 & 52 ± 4 & 50 ± 3 \\
			& & Klek      & 56 ± 8 & 55 ± 9 & 57 ± 5 & 54 ± 4 \\
			& & MACCS     & 55 ± 18 & 54 ± 18 & 57 ± 3 & 49 ± 6 \\
			& & Morgan    & 59 ± 10 & 54 ± 11 & 59 ± 4 & 55 ± 4 \\
			& & RDKit     & 55 ± 17 & 57 ± 17 & 59 ± 3 & 51 ± 6 \\
		& \multirow[t]{6}{*}{RandomForestClassifier}
			  & CDDD      & 66 ± 15 & 45 ± 18 & 61 ± 3 & 49 ± 12 \\
			& & ChemBERTa & 67 ± 15 & 41 ± 15 & 57 ± 4 & 47 ± 10 \\
			& & Klek      & 59 ± 19 & 54 ± 20 & 61 ± 3 & 50 ± 10 \\
			& & MACCS     & 63 ± 12 & 50 ± 13 & 61 ± 4 & 54 ± 6 \\
			& & Morgan    & 58 ± 21 & 55 ± 22 & 61 ± 3 & 49 ± 10 \\
			& & RDKit     & 66 ± 12 & 47 ± 12 & 61 ± 4 & 52 ± 7 \\
		& \multirow[t]{6}{*}{XGBClassifier}
			  & CDDD      & 64 ± 12 & 50 ± 15 & 61 ± 3 & 53 ± 7 \\
			& & ChemBERTa & 63 ± 13 & 46 ± 14 & 58 ± 3 & 50 ± 7 \\
			& & Klek      & 61 ± 14 & 54 ± 

### Statistical Tests

#### Primary vs Secondary

In [5]:
sign_level = 0.01
metrics = ['HarmRS', 'ROC AUC']

In [29]:
group_col = 'Dataset'
pairs = [('Primary', 'Secondary')]
index_cols = ['Model', 'Descriptors', 'Fold', 'PT', 'DPA']

for metric in metrics:

    dfs = []

    for file in glob.glob('../results/comparison/primary_secondary/*.tsv'):
        _, _, _, _, file_name = file.split('.')[2].split('/')
        pt_set, dpa_metric, model, descriptors, _ = file_name.split('_')
        df = (pl.from_pandas(read_df(file))
                .filter(pl.col('Metric') == metric)
                .rename({'Value': metric, 'Test_Fold': 'Fold'})
                .cast({'Fold': pl.String})
                .drop('Metric')
        )
        dfs.append(df)

    df = pl.concat(dfs)

    ds_df = compare_pairs(df, group_col=group_col, pairs=pairs, index_cols=index_cols,
                          sign_level=sign_level)

    print(f'< {metric} >')
    print(ds_df)

    ds_df = ds_df.to_pandas()
    ds_df.insert(loc=0, column='metric', value=([metric]*len(pairs)))

    write_df(ds_df, f'../results/analysis/wilcox/dataset_type_{metric}.tsv')

< HarmRS >
shape: (1, 7)
┌──────────────────────┬───────────────┬───────────┬─────────────┬─────────┬────────────┬──────────┐
│ null_hypothesis      ┆ null_rejected ┆ outcome   ┆ effect_size ┆ w_stat  ┆ p_value    ┆ r_stat   │
│ ---                  ┆ ---           ┆ ---       ┆ ---         ┆ ---     ┆ ---        ┆ ---      │
│ str                  ┆ bool          ┆ str       ┆ str         ┆ f64     ┆ f64        ┆ f64      │
╞══════════════════════╪═══════════════╪═══════════╪═════════════╪═════════╪════════════╪══════════╡
│ Primary == Secondary ┆ true          ┆ Primary > ┆ Large       ┆ 69897.0 ┆ 1.5844e-45 ┆ 0.574389 │
│                      ┆               ┆ Secondary ┆             ┆         ┆            ┆          │
└──────────────────────┴───────────────┴───────────┴─────────────┴─────────┴────────────┴──────────┘
< ROC AUC >
shape: (1, 7)
┌────────────────────────┬───────────────┬───────────┬─────────────┬──────────┬─────────┬──────────┐
│ null_hypothesis        ┆ null_rejected

#### Cred vs Card vs Cvas

In [27]:
group_col = 'PT'
pairs = [('Cred', 'Card'), ('Cred', 'Cvas'), ('Card', 'Cvas')]
index_cols = ['Model', 'Descriptors', 'Fold', 'Dataset', 'DPA']

for metric in metrics:

    dfs = []

    for file in glob.glob('../results/comparison/cred_card_cvas/*.tsv'):
        _, _, _, _, file_name = file.split('.')[2].split('/')
        dataset_type, dpa_metric, model, descriptors, _ = file_name.split('_')
        df = (pl.from_pandas(read_df(file))
                .filter(pl.col('Metric') == metric)
                .rename({'Value': metric, 'Test_Fold': 'Fold'})
                .cast({'Fold': pl.String})
                .drop('Metric')
        )
        dfs.append(df)

    df = pl.concat(dfs)

    pt_df = compare_pairs(df, group_col=group_col, pairs=pairs, index_cols=index_cols,
                      sign_level=sign_level)

    print(f'< {metric} >')
    print(pt_df)

    pt_df = pt_df.to_pandas()
    pt_df.insert(loc=0, column='metric', value=([metric]*len(pairs)))

    write_df(pt_df, f'../results/analysis/wilcox/pt_set_{metric}.tsv')

< HarmRS >
   metric null_hypothesis  null_rejected       outcome effect_size   w_stat  \
0  HarmRS    Cred == Card           True   Cred > Card       Small  59998.0   
1  HarmRS    Cred == Cvas          False  Cred == Cvas       Small  64303.0   
2  HarmRS    Card == Cvas          False  Card == Cvas  Negligible  71857.5   

    p_value    r_stat  
0  0.000326  0.178503  
1  0.016082  0.119559  
2  0.745583 -0.016122  
< ROC AUC >
    metric null_hypothesis  null_rejected       outcome effect_size   w_stat  \
0  ROC AUC    Cred == Card          False  Cred == Card  Negligible  69957.5   
1  ROC AUC    Cred == Cvas          False  Cred == Cvas  Negligible  71528.5   
2  ROC AUC    Card == Cvas          False  Card == Cvas  Negligible  69997.5   

    p_value    r_stat  
0  0.396298 -0.042137  
1  0.732590  0.016993  
2  0.402466  0.041590  


#### PRR vs ROR vs IC

In [6]:
group_col = 'DPA'
pairs = [('PRR', 'ROR'), ('PRR', 'IC'), ('ROR', 'IC')]
index_cols = ['Model', 'Descriptors', 'Fold', 'Dataset', 'PT']

for metric in metrics:

    dfs = []

    for file in glob.glob('../results/comparison/prr_ror_ic/*.tsv'):
        _, _, _, _, file_name = file.split('.')[2].split('/')
        dataset_type, pt_set, model, descriptors, _ = file_name.split('_')
        df = (pl.from_pandas(read_df(file))
                .filter(pl.col('Metric') == metric)
                .rename({'Value': metric, 'Test_Fold': 'Fold'})
                .cast({'Fold': pl.String})
                .drop('Metric')
        )
        dfs.append(df)

    df = pl.concat(dfs)

    dpa_df = compare_pairs(df, group_col=group_col, pairs=pairs, index_cols=index_cols, sign_level=sign_level)

    print(f'< {metric} >')
    print(dpa_df)

    dpa_df = dpa_df.to_pandas()
    dpa_df.insert(loc=0, column='metric', value=([metric]*len(pairs)))

    #write_df(dpa_df, f'../results/analysis/wilcox/dpa_metric_{metric}.tsv')

< HarmRS >
shape: (3, 7)
┌─────────────────┬───────────────┬───────────┬─────────────┬─────────┬────────────┬──────────┐
│ null_hypothesis ┆ null_rejected ┆ outcome   ┆ effect_size ┆ w_stat  ┆ p_value    ┆ r_stat   │
│ ---             ┆ ---           ┆ ---       ┆ ---         ┆ ---     ┆ ---        ┆ ---      │
│ str             ┆ bool          ┆ str       ┆ str         ┆ f64     ┆ f64        ┆ f64      │
╞═════════════════╪═══════════════╪═══════════╪═════════════╪═════════╪════════════╪══════════╡
│ PRR == ROR      ┆ true          ┆ PRR > ROR ┆ Medium      ┆ 39469.5 ┆ 3.4445e-20 ┆ 0.457576 │
│ PRR == IC       ┆ true          ┆ PRR > IC  ┆ Large       ┆ 3193.0  ┆ 1.3181e-82 ┆ 0.956281 │
│ ROR == IC       ┆ true          ┆ ROR > IC  ┆ Large       ┆ 15298.0 ┆ 4.8682e-57 ┆ 0.790539 │
└─────────────────┴───────────────┴───────────┴─────────────┴─────────┴────────────┴──────────┘
< ROC AUC >
shape: (3, 7)
┌─────────────────┬───────────────┬───────────┬─────────────┬─────────┬────────────┬─

#### Holm correction

In [32]:
from statsmodels.stats.multitest import multipletests

dfs = pl.concat([pl.DataFrame(read_df(file)) for file in glob.glob('../results/analysis/wilcox/*.tsv')])
dfs = dfs.sort('p_value')

rejected, p_adjusted, _, _ = multipletests(
    dfs['p_value'].to_numpy(),
    alpha=sign_level,
    method='holm',
    is_sorted=True
)

dfs = dfs.with_columns([
    pl.Series('p_value_adj', p_adjusted),
    pl.Series('null_rejected_adj', rejected),
])

write_df(dfs.to_pandas(), '../results/analysis/holm_correction.tsv')
dfs

metric,null_hypothesis,null_rejected,outcome,effect_size,w_stat,p_value,r_stat,p_value_adj,null_rejected_adj
str,str,bool,str,str,f64,f64,f64,f64,bool
"""HarmRS""","""PRR == IC""",true,"""PRR > IC""","""Large""",3193.0,1.3181e-82,0.956281,1.8454e-81,true
"""HarmRS""","""ROR == IC""",true,"""ROR > IC""","""Large""",15298.0,4.8682e-57,0.790539,6.3286e-56,true
"""HarmRS""","""Primary == Secondary""",true,"""Primary > Secondary""","""Large""",69897.0,1.5844e-45,0.574389,1.9013e-44,true
"""HarmRS""","""PRR == ROR""",true,"""PRR > ROR""","""Medium""",39469.5,3.4445e-20,0.457576,3.7890e-19,true
"""ROC AUC""","""ROR == IC""",true,"""ROR < IC""","""Medium""",44323.0,5.7360e-15,-0.38861,5.7360e-14,true
…,…,…,…,…,…,…,…,…,…
"""ROC AUC""","""Cred == Card""",false,"""Cred == Card""","""Negligible""",69957.5,0.396298,-0.042137,1.0,false
"""ROC AUC""","""Card == Cvas""",false,"""Card == Cvas""","""Negligible""",69997.5,0.402466,0.04159,1.0,false
"""ROC AUC""","""PRR == IC""",false,"""PRR == IC""","""Negligible""",71078.5,0.641158,0.023177,1.0,false


In [56]:
# LaTeX table
df = pl.from_pandas(read_df('../results/analysis/holm_correction.tsv'))

for metric in ['HarmRS', 'ROC AUC']:
    header_line = '\t\multicolumn{7}{*}{' + f"{metric}" + "} \\\\"
    print(header_line)
    sub_df = df.filter(pl.col('metric') == metric).drop('metric')
    for row in sub_df.iter_rows():
        null_hp, _, outcome, effect_size, w_stat, p_val, r_stat, p_val_adj, _ = row
        null_hp = null_hp.replace('==', '$\sim$')
        outcome = outcome.replace('==', '$\sim$')
        empty_lines = " " * (24 - len(null_hp))

        p_val = round_to_significant(p_val, 3)
        r_stat = round_to_significant(r_stat, 3)
        p_val_adj = round_to_significant(p_val_adj, 3)

        line = f"\t\t{null_hp} {empty_lines}& {int(w_stat)} & {p_val} & {r_stat} & {effect_size} & {p_val_adj} & {outcome} \\\\"
        print(line)

	\multicolumn{7}{*}{HarmRS} \\
		PRR $\sim$ IC            & 3193 & 1.32e-82 & 0.956 & Large & 1.85e-81 & PRR > IC \\
		ROR $\sim$ IC            & 15298 & 4.87e-57 & 0.791 & Large & 6.33e-56 & ROR > IC \\
		Primary $\sim$ Secondary & 69897 & 1.58e-45 & 0.574 & Large & 1.9e-44 & Primary > Secondary \\
		PRR $\sim$ ROR           & 39469 & 3.44e-20 & 0.458 & Medium & 3.79e-19 & PRR > ROR \\
		Cred $\sim$ Card         & 59998 & 0.000326 & 0.179 & Small & 0.00261 & Cred > Card \\
		Cred $\sim$ Cvas         & 64303 & 0.0161 & 0.12 & Small & 0.0965 & Cred $\sim$ Cvas \\
		Card $\sim$ Cvas         & 71857 & 0.746 & -0.0161 & Negligible & 1.0 & Card $\sim$ Cvas \\
	\multicolumn{7}{*}{ROC AUC} \\
		ROR $\sim$ IC            & 44323 & 5.74e-15 & -0.389 & Medium & 5.74e-14 & ROR < IC \\
		PRR $\sim$ ROR           & 45078 & 1.29e-14 & 0.383 & Medium & 1.16e-13 & PRR > ROR \\
		Primary $\sim$ Secondary & 142589 & 0.00116 & 0.132 & Small & 0.00812 & Primary > Secondary \\
		Cred $\sim$ Card         & 6

### Models' results on the selected variant

In [44]:
# Aggregate results on Primary-PRR-Cred variant

dfs = []

for file in glob.glob('../results/aggregated/primary_Cred_prr/*scores.tsv'):
    df = read_df(file)
    dfs.append(df)

df = pd.concat(dfs).reset_index(drop=True)

In [45]:
total_df = df[df['Kind'] == 'Total | Total']
sex_df = df[['Sex' in row['Kind'] for idx, row in df.iterrows()]].reset_index(drop=True)
age_df = df[['Age' in row['Kind'] for idx, row in df.iterrows()]].reset_index(drop=True)
wgt_df = df[['Weight' in row['Kind'] for idx, row in df.iterrows()]].reset_index(drop=True)

In [46]:
total_df = total_df.drop(columns=['TP', 'FP', 'FN', 'TN', 'Accuracy', 'Precision', 'Balanced Accuracy', 'GeomRS', 'F1 Score', 'Kind', 'MCC'])
total_df = pd.melt(total_df, id_vars=['Series', 'Test_Fold', 'Model', 'Descriptors'], value_vars=['Recall', 'Specificity', 'HarmRS', 'ROC AUC'], 
                   var_name='Metric', value_name='Value')
write_df(total_df, '../results/analysis/total.tsv')

In [47]:
sex_df.loc[:, 'Sex'] = sex_df['Kind'].apply(lambda desc: desc.split(' | ')[-1])
sex_df = sex_df.drop(columns=['TP', 'FP', 'FN', 'TN', 'Accuracy', 'Precision', 'Balanced Accuracy', 'GeomRS', 'F1 Score', 'Kind', 'MCC'])
sex_df = pd.melt(sex_df, id_vars=['Series', 'Test_Fold', 'Model', 'Descriptors', 'Sex'], value_vars=['Recall', 'Specificity', 'HarmRS', 'ROC AUC'], 
                   var_name='Metric', value_name='Value')
write_df(sex_df, '../results/analysis/sex.tsv')

In [48]:
age_df.loc[:, 'Age'] = age_df['Kind'].apply(lambda desc: desc.split(' | ')[-1])
age_df = age_df.drop(columns=['TP', 'FP', 'FN', 'TN', 'Accuracy', 'Precision', 'Balanced Accuracy', 'GeomRS', 'F1 Score', 'Kind', 'MCC'])
age_df = pd.melt(age_df, id_vars=['Series', 'Test_Fold', 'Model', 'Descriptors', 'Age'], value_vars=['Recall', 'Specificity', 'HarmRS', 'ROC AUC'], 
                   var_name='Metric', value_name='Value')
write_df(age_df, '../results/analysis/age.tsv')

In [49]:
wgt_df.loc[:, 'Weight'] = wgt_df['Kind'].apply(lambda desc: desc.split(' | ')[-1])
wgt_df = wgt_df.drop(columns=['TP', 'FP', 'FN', 'TN', 'Accuracy', 'Precision', 'Balanced Accuracy', 'GeomRS', 'F1 Score', 'Kind', 'MCC'])
wgt_df = pd.melt(wgt_df, id_vars=['Series', 'Test_Fold', 'Model', 'Descriptors', 'Weight'], value_vars=['Recall', 'Specificity', 'HarmRS', 'ROC AUC'], 
                   var_name='Metric', value_name='Value')
write_df(wgt_df, '../results/analysis/weight.tsv')

### Dataset characteristics

In [34]:
# Characterize all dataset variants

os.makedirs('../results/analysis/', exist_ok=True)

prms_df = []

dataset_dtype = pd.CategoricalDtype(categories=['Primary', 'Secondary'], ordered=True)
pt_dtype = pd.CategoricalDtype(categories=['Cred', 'Card', 'Cvas'], ordered=True)
dpa_dtype = pd.CategoricalDtype(categories=['PRR', 'ROR', 'IC'], ordered=True)

for file in glob.glob('../data/carbide/*/*/*.joblib'):
    dataset_type, pt_set, name = file.split('/')[-3:]
    dpa_metric = name.rstrip('.joblib').split('_')[-1]
    df = read_df(file)

    params_df = pd.DataFrame({
        'Dataset': [dataset_type.capitalize()],
        'PT': [pt_set.capitalize()],
        'DPA': [dpa_metric.upper()],
        'Size': [len(df)],
        'Unique SMILES': [df['SMILES'].nunique()],
        '% Toxic': [f'{df["Label"].mean()*100:.2f}'],
        'Conf. Score Toxic': [f'{df[df["Label"] == 1]["Label_weight"].mean()*100:.0f} | {df[df["Label"] == 1]["Label_weight"].std()*100:.0f}'],
        'Conf. Score Non-Toxic': [f'{df[df["Label"] == 0]["Label_weight"].mean()*100:.0f} | {df[df["Label"] == 0]["Label_weight"].std()*100:.0f}'],
        'EpS': [f'{df.groupby("SMILES").size().mean():.0f} | {df.groupby("SMILES").size().std():.0f}']
    })
    prms_df.append(params_df)

df = pd.concat(prms_df).reset_index(drop=True)

df = df.astype({
    'Dataset': dataset_dtype,
    'PT': pt_dtype,
    'DPA': dpa_dtype,
})

df = df.sort_values(by=['Dataset', 'PT', 'DPA']).reset_index(drop=True)

write_df(df, '../results/analysis/dataset_characterization.tsv')

In [43]:
# LaTeX table

df = read_df('../results/analysis/dataset_characterization.tsv')
df = pl.from_pandas(df)

for ds_type in ['Primary', 'Secondary']:
    ds_line = "\t\multirow[t]{9}{*}{" + ds_type + "}"
    print(ds_line)

    for pt_set in ['Cred', 'Card', 'Cvas']:
        pt_line = "\t\t& \multirow[t]{3}{*}{" + pt_set + "}"
        print(pt_line)

        for idx, dpa_metric in enumerate(['PRR', 'ROR', 'IC']):
            sub_df = df.filter(
                (pl.col("Dataset") == ds_type) &
                (pl.col("PT") == pt_set) &
                (pl.col("DPA") == dpa_metric)
            )

            n_comp = sub_df["Size"].item()
            n_unique = sub_df['Unique SMILES'].item()
            frac_toxic = sub_df['% Toxic'].item()
            cst_mean, cst_std = sub_df['Conf. Score Toxic'].item().split(" | ")
            csnt_mean, csnt_std = sub_df['Conf. Score Non-Toxic'].item().split(" | ")
            eps_mean, eps_std = sub_df['EpS'].item().split(" | ")

            if idx == 0:
                pre = "  &"
            else:
                pre = "& &"

            empty_lines = " " * (3 - len(dpa_metric))

            dpa_line = f"\t\t\t{pre} {dpa_metric} {empty_lines}& {n_comp} & {n_unique} & {frac_toxic:.2f} & {cst_mean} ± {cst_std} & {csnt_mean} ± {csnt_std} & {eps_mean} ± {eps_std} \\\\"
            print(dpa_line)


	\multirow[t]{9}{*}{Primary}
		& \multirow[t]{3}{*}{Cred}
			  & PRR & 26314 & 1158 & 57.70 & 28 ± 22 & 42 ± 29 & 23 ± 16 \\
			& & ROR & 26300 & 1158 & 54.49 & 24 ± 20 & 46 ± 31 & 23 ± 16 \\
			& & IC  & 26314 & 1158 & 57.71 & 38 ± 30 & 25 ± 16 & 23 ± 16 \\
		& \multirow[t]{3}{*}{Card}
			  & PRR & 28176 & 1193 & 58.68 & 28 ± 23 & 43 ± 29 & 24 ± 16 \\
			& & ROR & 28154 & 1193 & 55.26 & 25 ± 20 & 47 ± 32 & 24 ± 16 \\
			& & IC  & 28176 & 1193 & 58.68 & 38 ± 30 & 26 ± 17 & 24 ± 16 \\
		& \multirow[t]{3}{*}{Cvas}
			  & PRR & 32206 & 1251 & 58.30 & 32 ± 25 & 45 ± 30 & 26 ± 17 \\
			& & ROR & 32164 & 1250 & 54.16 & 27 ± 22 & 49 ± 32 & 26 ± 17 \\
			& & IC  & 32206 & 1251 & 58.31 & 41 ± 31 & 27 ± 19 & 26 ± 17 \\
	\multirow[t]{9}{*}{Secondary}
		& \multirow[t]{3}{*}{Cred}
			  & PRR & 37616 & 1788 & 61.46 & 29 ± 22 & 45 ± 29 & 21 ± 16 \\
			& & ROR & 37549 & 1786 & 56.27 & 26 ± 21 & 51 ± 32 & 21 ± 16 \\
			& & IC  & 37616 & 1788 & 61.46 & 42 ± 31 & 26 ± 17 & 21 ± 16 \\
		& \multirow[t]{3}{

### Class distribution analysis

In [50]:
"""
Analyse the class (and weights-adjusted) distribution of toxic class
At this point of data analysis, a colleague of mine suggested learning polars as an alternative to pandas, so the following section was my learning arc...
On a good note, the code is much more readable this way, as extensive groupby statements in pandas are kind of messy
"""

df = read_df('../data/carbide/primary/Cred/carbide_prr.joblib')
df = pl.from_pandas(df)

df = df.with_columns(
    Sex=pl.col('Signature').list.get(0),
    Age=pl.col('Signature').list.get(1),
    Weight=pl.col('Signature').list.get(2),
)

sdf = (df.group_by(['Sex', 'Label'])
    .agg([
        pl.col('Label_weight').mean().round(3).alias('Mean LW'),
        pl.len().alias('Entries')
    ])
    .with_columns([
        (pl.col('Mean LW') * pl.col('Entries')).round(3).alias('Contribution'),
        pl.col('Entries').sum().over('Sex').alias('Total Entries'),
        (pl.col('Mean LW') * pl.col('Entries')).sum().over('Sex').round(3).alias('Total Contribution')
    ])
    .with_columns([
        (pl.col('Entries') / pl.col('Total Entries')).round(3).alias('Frac.'),
        (pl.col('Contribution') / pl.col('Total Contribution')).round(3).alias('Adj. Frac.'),
    ])
    .rename({
    'Sex': 'Sub-category'
    })
)
sdf = sdf.with_columns(Category=pl.lit('Sex'))

adf = (df.group_by(['Age', 'Label'])
    .agg([
        pl.col('Label_weight').mean().alias('Mean LW'),
        pl.len().alias('Entries')
    ])
    .with_columns([
        (pl.col('Mean LW') * pl.col('Entries')).alias('Contribution'),
        pl.col('Entries').sum().over('Age').alias('Total Entries'),
        (pl.col('Mean LW') * pl.col('Entries')).sum().over('Age').alias('Total Contribution')
    ])
    .with_columns([
        (pl.col('Entries') / pl.col('Total Entries')).alias('Frac.'),
        (pl.col('Contribution') / pl.col('Total Contribution')).alias('Adj. Frac.')
    ])
    .rename({
    'Age': 'Sub-category'
    })
)
adf = adf.with_columns(Category=pl.lit('Age'))

wdf = (df.group_by(['Weight', 'Label'])
    .agg([
        pl.col('Label_weight').mean().alias('Mean LW'),
        pl.len().alias('Entries')
    ])
    .with_columns([
        (pl.col('Mean LW') * pl.col('Entries')).alias('Contribution'),
        pl.col('Entries').sum().over('Weight').alias('Total Entries'),
        (pl.col('Mean LW') * pl.col('Entries')).sum().over('Weight').alias('Total Contribution')
    ])
    .with_columns([
        (pl.col('Entries') / pl.col('Total Entries')).alias('Frac.'),
        (pl.col('Contribution') / pl.col('Total Contribution')).alias('Adj. Frac.')
    ])
    .rename({
    'Weight': 'Sub-category'
    })
)
wdf = wdf.with_columns(Category=pl.lit('Weight'))

class_analysis = pl.concat([sdf, adf, wdf])

In [51]:
class_analysis = (class_analysis
    .with_columns([
        pl.col('Mean LW').round(3).alias('Mean LW'),
        pl.col('Contribution').round(3).alias('Contribution'),
        (pl.col('Frac.') * 100).round(3).alias('Frac.'),
        (pl.col('Adj. Frac.') * 100).round(3).alias('Adj. Frac.')])
    .rename({
        'Label': 'Class'})
    .with_columns(
        pl.col('Class').cast(pl.String).replace({
            '0': 'Non-Toxic',
            '1': 'Toxic'}))
)

class_analysis.write_csv('../results/analysis/class_analysis.tsv', separator='\t')

In [59]:
# LaTeX table

for number, category in zip([6, 10, 8], ['Sex', 'Age', 'Weight']):
    ds_line = "\t\multirow[t]{" + str(number) + "}{*}{" + category + "}"
    print(ds_line)

    if category == 'Sex':
        iter_over = ['Male', 'Female', 'Unknown']
    elif category == 'Age':
        iter_over = ['Children', 'Adolescent', 'Adult', 'Elderly', 'Unknown']
    elif category == 'Weight':
        iter_over = ['Low', 'Average', 'High', 'Unknown']

    for iter in iter_over:
        sub_line = "\t\t& \multirow[t]{2}{*}{" + iter + "}"
        print(sub_line)
        for idx, tx in enumerate(['Toxic', 'Non-Toxic']):
            sub_df = class_analysis.filter(
                (pl.col("Category") == category) &
                (pl.col("Sub-category") == iter) &
                (pl.col("Class") == tx)
            )
            entries = sub_df['Entries'].item()
            mean_lw = sub_df['Mean LW'].item()
            frac = sub_df['Frac.'].item()
            adj_frac = sub_df['Adj. Frac.'].item()

            if idx == 0:
                pre = "  &"
            else:
                pre = "& &"

            empty_lines = " " * (9 - len(tx))

            line = f"\t\t\t{pre} {tx} {empty_lines}& {entries} & {mean_lw:.3f} & {frac:.2f} & {adj_frac:.2f} \\\\"
            print(line)

	\multirow[t]{6}{*}{Sex}
		& \multirow[t]{2}{*}{Male}
			  & Toxic     & 4313 & 0.265 & 54.60 & 43.00 \\
			& & Non-Toxic & 3589 & 0.423 & 45.40 & 57.00 \\
		& \multirow[t]{2}{*}{Female}
			  & Toxic     & 4885 & 0.263 & 59.50 & 48.50 \\
			& & Non-Toxic & 3330 & 0.409 & 40.50 & 51.50 \\
		& \multirow[t]{2}{*}{Unknown}
			  & Toxic     & 5985 & 0.294 & 58.70 & 48.90 \\
			& & Non-Toxic & 4212 & 0.436 & 41.30 & 51.10 \\
	\multirow[t]{10}{*}{Age}
		& \multirow[t]{2}{*}{Children}
			  & Toxic     & 1297 & 0.166 & 67.34 & 51.29 \\
			& & Non-Toxic & 629 & 0.324 & 32.66 & 48.71 \\
		& \multirow[t]{2}{*}{Adolescent}
			  & Toxic     & 1316 & 0.181 & 69.70 & 55.46 \\
			& & Non-Toxic & 572 & 0.334 & 30.30 & 44.54 \\
		& \multirow[t]{2}{*}{Adult}
			  & Toxic     & 4075 & 0.271 & 56.64 & 45.80 \\
			& & Non-Toxic & 3119 & 0.420 & 43.36 & 54.20 \\
		& \multirow[t]{2}{*}{Elderly}
			  & Toxic     & 3324 & 0.287 & 51.96 & 42.21 \\
			& & Non-Toxic & 3073 & 0.425 & 48.04 & 57.79 \\
		& \multirow[t

#### RS ratio analysis

In [60]:
# Calculate the Recall/Specificity Ratio and compare with toxic class distribution

cl_df = pl.read_csv('../results/analysis/class_analysis.tsv', separator='\t')
cl_df = cl_df.filter(pl.col('Class') != 'Non-Toxic')

tot_df = read_df('../results/analysis/total.tsv')
sex_df = read_df('../results/analysis/sex.tsv')
age_df = read_df('../results/analysis/age.tsv')
wgt_df = read_df('../results/analysis/weight.tsv')

sex_df['Stratification'] = 'Sex'
sex_df = sex_df.rename(columns={'Sex':'Cat'})

age_df['Stratification'] = 'Age'
age_df = age_df.rename(columns={'Age':'Cat'})

wgt_df['Stratification'] = 'Weight'
wgt_df = wgt_df.rename(columns={'Weight':'Cat'})

df = pd.concat([sex_df, age_df, wgt_df], axis=0).reset_index(drop=True)

res_df = pl.from_pandas(df)
res_df = res_df.filter(pl.col('Series') == "Unweighted").drop(['Series', 'Test_Fold', 'Model', 'Descriptors'])

res_df_proc = (res_df.group_by(['Stratification', 'Cat', 'Metric'])
    .agg(pl.col('Value').mean())
    .filter(pl.col('Metric').is_in(['Recall', 'Specificity']))
    .pivot(index=['Stratification', 'Cat'], on='Metric')
    .with_columns((pl.col('Recall') / pl.col('Specificity')).alias('RS ratio').round(3))
    .rename({'Stratification': 'Category', 'Cat': 'Sub-category'})
)

df = res_df_proc.join(cl_df, on=['Category', 'Sub-category']).drop('Class')

df = df.with_columns([
    (pl.col('Frac.') / 100).alias('Fraction Toxic'),
    (pl.col('Adj. Frac.') / 100).alias('Adjusted Fraction Toxic'),
])
df = df.drop(['Frac.', 'Adj. Frac.'])

df.write_csv('../results/analysis/RS_ratio_analysis.tsv', separator='\t')

### Predictions differences

In [ ]:
# Aggregate the results of prediction differences

differences = [joblib.load(file) for file in glob.glob('../results/differences/*.joblib')]

# aggregate all predictions per SMILES
ddc = defaultdict(list)

for diff in differences:
    for fold, results in diff.items():
        for smiles, array in results.items():
            ddc[smiles].append(array)

# calculate mean per demographic group
ddc_cc = {}
for smiles, arrays in ddc.items():
    ddc_cc[smiles] = np.mean(ddc[smiles], axis=0)

# build a DataFrame
idc_map = {
    0: ('Sex', 'Male'),
    1: ('Sex', 'Female'),
    2: ('Age', 'Children'),
    3: ('Age', 'Adolescent'),
    4: ('Age', 'Adult'),
    5: ('Age', 'Elderly'),
    6: ('Weight', 'Low'),
    7: ('Weight', 'Average'),
    8: ('Weight', 'High')
}

df_dc = defaultdict(list)

for smiles, array in ddc_cc.items():
    for idx, assignment in idc_map.items():
        df_dc['SMILES'].append(smiles)
        df_dc['Category'].append(assignment[0])
        df_dc['Sub-category'].append(assignment[1])
        df_dc['Difference'].append(np.round(array[0][idx], 5))

df = pd.DataFrame(df_dc)
write_df(df, '../results/analysis/prediction_differences.tsv')

### Models' predictions

In [ ]:
# Combine model prediction analysis with class and RS ratio analysis.

sex_df = read_df('../results/analysis/sex.tsv')
age_df = read_df('../results/analysis/age.tsv')
wgt_df = read_df('../results/analysis/weight.tsv')

sex_df['Stratification'] = 'Sex'
sex_df = sex_df.rename(columns={'Sex':'Category'})

age_df['Stratification'] = 'Age'
age_df = age_df.rename(columns={'Age':'Category'})

wgt_df['Stratification'] = 'Weight'
wgt_df = wgt_df.rename(columns={'Weight':'Category'})

df = pd.concat([sex_df, age_df, wgt_df], axis=0).reset_index(drop=True)
df = pl.from_pandas(df).rename({'Category': 'Sub-category'})

df_ = pl.read_csv('../results/analysis/RS_ratio_analysis.tsv', separator='\t').rename({'Category': 'Stratification'})
df_ = df_.drop(['Entries', 'Mean LW', 'Fraction Toxic', 'RS ratio', 'Recall', 'Specificity'])
df_ = (df_.rename({'Adjusted Fraction Toxic': 'AFT'})
       .unpivot(on='AFT', index=['Stratification', 'Sub-category'], value_name='Value', variable_name='Metric'))

df = (df.drop(['Test_Fold', 'Model', 'Descriptors', 'Series'])
      .select(['Stratification', 'Sub-category', 'Metric', 'Value']))

df = pl.concat([df, df_])
df.write_csv('../results/analysis/models_metrics.tsv', separator='\t')